In [1]:
from aoc import read_file

In [188]:
lines = read_file('20221207.txt').split('\n')

commands = [line.split(' ') for line in lines]

In [189]:
import json

def parse_commands(commands):
    i = 1

    root = {
        'name': '/',
        'path': '/',
        'type': 'dir',
        'contents': {}
    }

    dir_stack = [root]

    current_dir = root

    while i < len(lines):
        command = commands[i]
        if command[0] != '$':
            raise Exception('Should see command: {lines[index]}')
        if command[1] == 'cd':
            if command[2] == '..':
                current_dir = dir_stack.pop()
            else:
                directory_name = command[2]
                dir_stack.append(current_dir)
                current_dir = current_dir['contents'][directory_name]
        elif command[1] == 'ls':
            while i+1 < len(commands) and commands[i+1][0] != '$':
                i += 1
                if commands[i][0] == 'dir':
                    directory_name = commands[i][1]
                    current_dir['contents'][directory_name] = {
                        'name': directory_name,
                        'path': current_dir['path'] + '/' + directory_name,
                        'type': 'dir',
                        'contents': {}
                    }
                else:
                    size = int(commands[i][0])
                    file_name = commands[i][1]
                    current_dir['contents'][file_name] = {
                        'name': file_name,
                        'type': 'file',
                        'size': size
                    }
        i += 1

    return root

In [203]:
directory_tree = parse_commands(commands)

In [220]:
def get_directory_sizes(directory):
    sizes = {
        'name': directory['name'],
        'path': directory['path'],
        'contents': {}
    }
    
    dir_size = 0
    
    for name, content_file in directory['contents'].items():
        if content_file['type'] == 'file':
            dir_size += content_file['size']
        else:
            subdir_size = get_directory_sizes(content_file)
            sizes['contents'][name] = subdir_size
            dir_size += subdir_size['size']

    sizes['size'] = dir_size
    return sizes

In [247]:
from functools import reduce

def get_directory_sizes2(directory):
    ret = {}
    
    file_sizes = sum([content_file['size'] for content_file in directory['contents'].values() if content_file['type'] == 'file'])
    dir_sizes = [get_directory_sizes2(content_file) for content_file in directory['contents'].values() if content_file['type'] == 'dir']
    if len(dir_sizes) > 0:
        ret = reduce(lambda a, b: {**a, **b}, dir_sizes)
        ret[directory['path']] = file_sizes + sum([size for size in ret.values()])
    else:
        ret[directory['path']] = file_sizes
    
    return ret

get_directory_sizes2(directory_tree)

{'//fmfnpm/fgtqvq': 618418,
 '//fmfnpm/rvnwwfq': 76914,
 '//fmfnpm/wrzcjwc/fwdwq': 120795,
 '//fmfnpm/wrzcjwc/lddhdslh/gzj/qzgsswr': 155330,
 '//fmfnpm/wrzcjwc/lddhdslh/gzj': 155330,
 '//fmfnpm/wrzcjwc/lddhdslh': 603690,
 '//fmfnpm/wrzcjwc/mjp': 458712,
 '//fmfnpm/wrzcjwc': 1976775,
 '//fmfnpm/zlpmfh': 189296,
 '//fmfnpm': 4903164,
 '//gwlwp/dwcrnbj/jbtfslcn': 195255,
 '//gwlwp/dwcrnbj/lgbglc/qdpjss': 151905,
 '//gwlwp/dwcrnbj/lgbglc': 1409518,
 '//gwlwp/dwcrnbj/sbffqq': 29842,
 '//gwlwp/dwcrnbj/zhvn/clpcg': 299295,
 '//gwlwp/dwcrnbj/zhvn/gswvch/fzb/qzgsswr': 198805,
 '//gwlwp/dwcrnbj/zhvn/gswvch/fzb': 198805,
 '//gwlwp/dwcrnbj/zhvn/gswvch/mdrlrtl': 154648,
 '//gwlwp/dwcrnbj/zhvn/gswvch': 552258,
 '//gwlwp/dwcrnbj/zhvn/lgmfhnq': 303154,
 '//gwlwp/dwcrnbj/zhvn': 2227003,
 '//gwlwp/dwcrnbj': 6342883,
 '//gwlwp/fmfnpm/fzb/fzb/gwtslbr': 224479,
 '//gwlwp/fmfnpm/fzb/fzb': 224479,
 '//gwlwp/fmfnpm/fzb': 448958,
 '//gwlwp/fmfnpm/svtswb/fmfnpm': 318326,
 '//gwlwp/fmfnpm/svtswb/msfd': 86195,
 '

In [243]:
directory_sizes = get_directory_sizes(directory_tree)

In [244]:
def get_directory_size_map(directory_sizes):
    ret = {
        directory_sizes['path']: directory_sizes['size']
    }
    for sub_dir in directory_sizes['contents'].values():
        ret = {**ret, **get_directory_size_map2(sub_dir)}
    return ret

In [245]:
size_map = get_directory_size_map2(directory_sizes)
size_map

{'/': 46090134,
 '//fmfnpm': 2943317,
 '//fmfnpm/fgtqvq': 618418,
 '//fmfnpm/rvnwwfq': 76914,
 '//fmfnpm/wrzcjwc': 1510785,
 '//fmfnpm/wrzcjwc/fwdwq': 120795,
 '//fmfnpm/wrzcjwc/lddhdslh': 448360,
 '//fmfnpm/wrzcjwc/lddhdslh/gzj': 155330,
 '//fmfnpm/wrzcjwc/lddhdslh/gzj/qzgsswr': 155330,
 '//fmfnpm/wrzcjwc/mjp': 458712,
 '//fmfnpm/zlpmfh': 189296,
 '//gwlwp': 34257857,
 '//gwlwp/dwcrnbj': 3732950,
 '//gwlwp/dwcrnbj/jbtfslcn': 195255,
 '//gwlwp/dwcrnbj/lgbglc': 1409518,
 '//gwlwp/dwcrnbj/lgbglc/qdpjss': 151905,
 '//gwlwp/dwcrnbj/sbffqq': 29842,
 '//gwlwp/dwcrnbj/zhvn': 1475940,
 '//gwlwp/dwcrnbj/zhvn/clpcg': 299295,
 '//gwlwp/dwcrnbj/zhvn/gswvch': 353453,
 '//gwlwp/dwcrnbj/zhvn/gswvch/fzb': 198805,
 '//gwlwp/dwcrnbj/zhvn/gswvch/fzb/qzgsswr': 198805,
 '//gwlwp/dwcrnbj/zhvn/gswvch/mdrlrtl': 154648,
 '//gwlwp/dwcrnbj/zhvn/lgmfhnq': 303154,
 '//gwlwp/fmfnpm': 1389190,
 '//gwlwp/fmfnpm/fzb': 224479,
 '//gwlwp/fmfnpm/fzb/fzb': 224479,
 '//gwlwp/fmfnpm/fzb/fzb/gwtslbr': 224479,
 '//gwlwp/fmfnp

In [227]:
sum([size for path, size in size_map.items() if size <= 100000])

1491614

In [228]:
available_space = 40000000

used_space = size_map['/']

required_space = used_space - available_space

for size in sorted([size for path, size in size_map.items()]):
    if size < required_space:
        continue
    else:
        print(size)
        break


6400111
